In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-19 21:02:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  6.17MB/s    in 0.2s    

2021-06-19 21:02:03 (6.17 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

In [7]:
#Filter vine_df to create another dataframe where there are 20 or more total votes
from pyspark.sql.functions import concat, col, lit
vine_filtered = vine_df.filter("total_votes>=20").show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R2JK5Y8D5MXAGP|          1|            5|        121|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1H5Y9Z2DHRNVO|          1|            4|         95|   N|                Y|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|     

In [8]:
#filter the data to create a DataFrame where the percentage of helpful_votes is equal to or greater than 50%. 
vine_per = vine_df.withColumn("voting_perc", vine_df['helpful_votes'] / vine_df['total_votes'])
vine_50 = vine_per.where(col('voting_perc') >= 0.5)

In [9]:
vine_50.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       voting_perc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|0.9761904761904762|
|R19VVIUT4BZCMT|          5|            2|          2|   N|                Y|               1.0|
|R34EZZ68VYPHO0|          1|            1|          1|   N|                Y|               1.0|
|R12UBZT87UX3AP|          5|            2|          2|   N|                Y|               1.0|
| RT12RQFU0V2AX|          5|            1|          1|   N|                Y|               1.0|
|R2K0YQPV5W2SJ1|          4|            2|          2|   N|                Y|               1.0|
|R3IF1H9L7WSGQD|          5|            1|          1|   N|                N|               1.0|
| R5I2YBT9J85FF|          5|  

In [10]:
#The data is filtered to create a DataFrame or table where there is a Vine review.
vine_paid = vine_50.where(col('vine') == 'Y')
vine_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       voting_perc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R323MJVCE76RHX|          5|            3|          6|   Y|                N|               0.5|
|R3BOQTLUU3Y4L8|          4|           42|         55|   Y|                N|0.7636363636363637|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|0.8898876404494382|
|R2MUPC17C2FAA3|          4|           11|         14|   Y|                N|0.7857142857142857|
| R19DZ4ZILR75G|          4|            5|          9|   Y|                N|0.5555555555555556|
|R37RHDWFZYGHQS|          5|            6|         10|   Y|                N|               0.6|
| REPU28WG1VZUE|          5|          242|        281|   Y|                N|0.8612099644128114|
|R1D6REC9HPJVQY|          4|  

In [11]:
#The data is filtered to create a DataFrame or table where there isn’t a Vine review.
vine_unpaid = vine_50.where(col('vine') == 'N')
vine_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       voting_perc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|0.9761904761904762|
|R19VVIUT4BZCMT|          5|            2|          2|   N|                Y|               1.0|
|R34EZZ68VYPHO0|          1|            1|          1|   N|                Y|               1.0|
|R12UBZT87UX3AP|          5|            2|          2|   N|                Y|               1.0|
| RT12RQFU0V2AX|          5|            1|          1|   N|                Y|               1.0|
|R2K0YQPV5W2SJ1|          4|            2|          2|   N|                Y|               1.0|
|R3IF1H9L7WSGQD|          5|            1|          1|   N|                N|               1.0|
| R5I2YBT9J85FF|          5|  

In [12]:
#Total reviews
paid_reviews = vine_paid.count()
unpaid_reviews = vine_unpaid.count()

#Total 5 star reviews
paid_5star = vine_paid.where(col('star_rating')==5).count()
unpaid_5star = vine_unpaid.where(col('star_rating')==5).count()

#Percentage of 5 star reviews
paid_5star_perc = (paid_5star / paid_reviews)*100
unpaid_5star_perc = (unpaid_5star / unpaid_reviews)*100

In [13]:
print('Total Paid reviews:', paid_reviews, '\n'
      'Total unpaid reviews:', unpaid_reviews, '\n'
      'Paid 5 star reviews:', paid_5star, '\n'
      'Unpaid 5 star reviews:', unpaid_5star, '\n'
      'Paid 5 star reviews percentage:', (paid_5star / paid_reviews)*100, '\n'
      'Unpaid 5 star reviews percentage:',  (unpaid_5star/unpaid_reviews)*100)

Total Paid reviews: 12 
Total unpaid reviews: 30029 
Paid 5 star reviews: 4 
Unpaid 5 star reviews: 14511 
Paid 5 star reviews percentage: 33.33333333333333 
Unpaid 5 star reviews percentage: 48.32328748876086
